In [ ]:
# Dependencies
import requests
import json
import pandas as pd
import ast

In [ ]:
# Api Call to RAWG gaming database.
# Running this will take over an hour... you've been warned.
dictionary_list = []
for i in range(2500):
    url = f'https://api.rawg.io/api/games?key=087fec04dc43483d8057bacd2bd5be04&page_size=40&page={i+1}'
    response = requests.get(url).json()
    dictionary_list = dictionary_list + response['results']
    print (f'Page {i+1} done')

In [ ]:
df = pd.json_normalize(dictionary_list)

In [ ]:
df.to_csv('data.csv')

In [ ]:
df = pd.read_csv ('data.csv')

In [ ]:
#Api call to Steam Spy dataset.
name = []
postive = []
negative = []
owners = []
price = []
publisher = []
for i in range(51):
    url = f'https://steamspy.com/api.php?request=all&page={i+1}'
    response = requests.get(url).json()
    for j in response:
        name.append(response[j]['name'])
        postive.append(response[j]['positive'])
        negative.append(response[j]['negative'])
        owners.append(response[j]['owners'])
        price.append(response[j]['initialprice'])
        publisher.append(response[j]['publisher'])
    #dictionary_list = dictionary_list + response['results']
    print (f'Page {i+1} done')

In [ ]:
# Creating df from result lists from API call.
steam_df = pd.DataFrame({'name': name, 'postive' : postive, 'negative' : negative, 'owners' : owners, 'price' : price, 'publisher' : publisher})
steam_df.head(2)

In [ ]:
# Writing to save data
steam_df.to_csv('steam_data.csv')

In [ ]:
# If you already have dataset.
steam_df = pd.read_csv ('steam_data.csv')

In [ ]:
# Merged to combine steam data with original dateframe.
merged_df = pd.merge(df, steam_df, how='inner', left_on = 'name', right_on = 'name')

In [ ]:
merged_df.to_csv('merged_data.csv')

In [ ]:
merged_df = pd.read_csv ('merged_data.csv')

In [ ]:
# Cleaning unecessary columns.
drop_columns = ['id', 'slug', 'tba',
       'rating', 'rating_top', 'ratings', 'ratings_count',
       'reviews_text_count', 'added', 'metacritic', 'playtime',
       'suggestions_count', 'updated', 'user_game', 'reviews_count',
       'saturated_color', 'dominant_color', 
       'stores', 'clip', 'short_screenshots',
       'added_by_status.yet', 'added_by_status.owned',
       'added_by_status.beaten', 'added_by_status.toplay',
       'added_by_status.dropped', 'added_by_status.playing', 
       'esrb_rating.name', 'esrb_rating.slug', 'esrb_rating',
       'community_rating', 'added_by_status', 'platforms', 'publisher']
merged_df.drop(drop_columns, inplace=True, axis=1)

#'Unnamed: 0', 'Unnamed: 0.1',

In [ ]:
# Calculating the total reviews to then find total positive reviews as a % (our target)
merged_df['total_reviews'] = merged_df['postive'] + merged_df['negative']
merged_df['review_score'] = merged_df['postive'] / merged_df['total_reviews']

In [ ]:
drop_columns2 = ['postive', 'negative']
merged_df.drop(drop_columns2, inplace=True, axis=1)

In [ ]:
drop_columns3 = ['Unnamed: 0_y', 'Unnamed: 0_x']
merged_df.drop(drop_columns3, inplace=True, axis=1)

In [ ]:
# Renaming combined dataset back to df.
df = merged_df

In [ ]:
# Making sure the format is correctly datetime, then splitting the release month (the value we thought would be most relevant to it's review score).
df['released'] = pd.to_datetime(df['released'], format = '%Y-%m-%dT', errors = 'coerce')
df['released_month'] = df['released'].dt.month

In [ ]:
# Declaring the lists needed to gather the data.
# The _list variable will store the complete data from each stated column.
# Each game has their own list of Genres, Tags, and Original Release Platforms
# The _names for genre and platforms are used to collect the unique values for making the new columns
# The _count for tags are used to see which tags are used most frequently
platforms_list = []
platforms_name = []
tags_list =[]
tags_count = []
genres_list = []
genres_names = []

# This reads through the entire dataframe.
# Saves each column's value and reads the value properly as a dictionary.
for i in range(len(df.index)):
    game_plat = df.parent_platforms[i]
    game_tag = df.tags[i]
    game_name = df.genres[i]

    game_plat_dict = ast.literal_eval(game_plat)
    game_tag_dict = ast.literal_eval(game_tag)
    game_name_dict = ast.literal_eval(game_name)

    # These short-lived variables are constantly tracking each games indvidual values and then adding them to the main lists.
    platforms = []
    genres = []
    tags = []

    # Because each column (genre, tag, and platform) was organized into a dictionary of variable length, these loops
    # are used to read through them and grab the relevant 'name' values.
    for j in range(len(game_plat_dict)):
        platforms.append(game_plat_dict[j]['platform']['name'])
        if game_plat_dict[j]['platform']['name'] not in platforms_name:
            platforms_name.append(game_plat_dict[j]['platform']['name'])
    platforms.sort()
    platforms_list.append(platforms)

    for k in range(len(game_tag_dict)):
        tags.append(game_tag_dict[k]['name'])
        tags_count.append(game_tag_dict[k]['name'])
    tags.sort()
    tags_list.append(tags)

    for l in range(len(game_name_dict)):
        genres.append(game_name_dict[l]['name'])
        if game_name_dict[l]['name'] not in genres_names:
            genres_names.append(game_name_dict[l]['name'])
    genres.sort()
    genres_names.sort(reverse=True)
    genres_list.append(genres)

In [ ]:
# The tags were each collected into one giant pool. This then sorts them and counts them.
from collections import Counter
tags_dict = Counter(tags_count)

sorted_tags = {}
sorted_keys = sorted(tags_dict, key=tags_dict.get, reverse=True) 

for w in sorted_keys:
    sorted_tags[w] = tags_dict[w]

# Here the sorted keys are then filtered.  First the top 25 most common were selected, but anything directly
# Steam related was removed as it did not seem a relevant selling or scoring point to a game.
# RPG tag was removed as well as it is already tracked as a genre.
tag_keys = list(sorted_tags.keys())
filtered_tag_keys = tag_keys[:25]
for key in filtered_tag_keys:
    if "steam" in key.lower():
        filtered_tag_keys.remove(key)
filtered_tag_keys.remove('RPG')

In [ ]:
# Insert the columns so games data could displayed.
for platform in platforms_name:
    df.insert(1,f"platform_{platform.lower()}",0)

for tag in filtered_tag_keys:
    df.insert(1,f"tag_{tag.lower()}",0)

for genre in genres_names:
    df.insert(1,f"genre_{genre.lower()}",0)

In [ ]:
# Finally reading the organized data back into each game.
# Each of the possible genres, tags, and platforms are added as columns to dataset and games populate the values appropriately.
for i in range(len(platforms_list)):
    for j in range(len(platforms_list[i])):
        for k in range(len(platforms_name)):
            if platforms_list[i][j] == platforms_name[k]:
                df.at[i,f'platform_{platforms_name[k].lower()}'] = 1

for i in range(len(tags_list)):
    for j in range(len(tags_list[i])):
        for k in range(len(filtered_tag_keys)):
            if tags_list[i][j] == filtered_tag_keys[k]:
                df.at[i,f"tag_{filtered_tag_keys[k].lower()}"] = 1


for i in range(len(genres_list)):
    for j in range(len(genres_list[i])):
        for k in range(len(genres_names)):
            if genres_list[i][j] == genres_names[k]:
                df.at[i,f'genre_{genres_names[k].lower()}'] = 1

In [ ]:
# Dropping the parent columns now that we extracted the necessary dictionary values.
df = df.drop(columns=['genres', 'tags','parent_platforms','released'])

# There is no ESRB rating 0. We are using it for games that lacked a rating.
df['esrb_rating.id'] = df['esrb_rating.id'].fillna(0)

df.drop_duplicates()

In [ ]:
# Checking how many null values are left, important to remove before attempting machine learning.
df_columns = df.columns.tolist()
for column in df_columns:
    if df[column].isnull().sum() != 0:
        print (column)
        print(df[column].isnull().sum())

In [ ]:
# For Boris' eyes only.
df.to_csv('boris_data.csv')

In [ ]:
df2 = pd.get_dummies(df, columns=["owners"],drop_first=True)